In [87]:
import sys
if 'google.colab' in sys.modules:
    print ("setting tensorflow version in colab")
    %tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.1.0'

In [103]:
import os
import lib
import random
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import lib as mlutils


from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, concatenate, Input, Conv2D,Flatten
from tensorflow.keras.backend import clear_session

from importlib import reload
reload(lib)

import warnings
warnings.filterwarnings('ignore')

In [104]:
data_dir = "/home/user/DeepLearning/projects/datasets/leaf_classification/"

### Read data from the .csv file 

In [105]:
split=0.7
X_train_f, _, X_train_ri, y_train, X_test_f, _, X_test_ri, y_test, species, num_classes, _, _ = lib.get_splitted_data(data_dir=data_dir, check_id_sets=True, verbose=1, use_resize_images=True, split=split)

The intersection between train and test set is 0
There are 99 classes for the classification task.


### Traget onehot 

In [106]:
y_train_oh = np.eye(99)[y_train]#num_classes
y_test_oh  = np.eye(num_classes)[y_test]

In [107]:
X_train_ri = X_train_ri.reshape((X_train_ri.shape[0], X_train_ri.shape[1], X_train_ri.shape[2], 1))

In [108]:
X_test_ri = X_test_ri.reshape((X_test_ri.shape[0], X_test_ri.shape[1], X_test_ri.shape[2], 1))

In [109]:
X_train_f.shape, X_train_ri.shape, y_train_oh.shape

((693, 194), (693, 128, 128, 1), (693, 99))

In [117]:
X_test_f.shape, X_test_ri.shape, y_test_oh.shape

((297, 194), (297, 128, 128, 1), (297, 99))

In [118]:
x_trainif = np.r_[[i.flatten() for i in X_train_ri]]
x_testif  = np.r_[[i.flatten() for i in X_test_ri]]

### Build a model 

In [141]:
def get_conv_model(num_classes, img_size=32, compile=True):
    print("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    layers = tf.keras.layers.Conv2D(2,(3,3), activation="relu")(inputs) 
    layers = tf.keras.layers.BatchNormalization()(layers)
    layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.Dense(100, activation=tf.nn.relu)(layers)
    layers = tf.keras.layers.BatchNormalization()(layers)
    layers = tf.keras.layers.Dense(100, activation=tf.nn.relu)(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax, name="output_1")(layers)
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    if compile:
        model.compile(optimizer='RMSprop',loss='categorical_crossentropy' ,metrics=['accuracy'] )
    return model

In [143]:
#instantiate the model
num_classes = len( y_test_oh[1])
model = get_conv_model(num_classes,img_size=128)
model.summary()

using 99 classes
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 126, 126, 2)       20        
_________________________________________________________________
batch_normalization_17 (Batc (None, 126, 126, 2)       8         
_________________________________________________________________
flatten_10 (Flatten)         (None, 31752)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 100)               3175300   
_________________________________________________________________
batch_normalization_18 (Batc (None, 100)               400       
_________________________________________________________________
dense_21 (Dense)             (None, 100) 

### Train fuction

In [144]:
def train(model, batch_size, epochs, model_name=""):
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/"+model_name+"_"+"{}".format(time()))
    model.reset_states()
    model.fit(X_train_ri.astype(np.float32), y_train_oh.astype(int), epochs=epochs, callbacks=[tensorboard], batch_size=batch_size, validation_data=( X_test_ri, y_test_oh))
    metrics = model.evaluate( X_test_ri.astype(np.float32), y_test_oh.astype(int))
    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [145]:
train(model, batch_size=8, epochs=10, model_name="model_A")

Train on 693 samples, validate on 297 samples
Epoch 1/10
693/693 [==============================] - 5s 8ms/sample - loss: 4.3339 - accuracy: 0.0577 - val_loss: 4.5877 - val_accuracy: 0.0505
Epoch 2/10
693/693 [==============================] - 4s 6ms/sample - loss: 3.3432 - accuracy: 0.2266 - val_loss: 7.7295 - val_accuracy: 0.0303
Epoch 3/10
693/693 [==============================] - 4s 6ms/sample - loss: 2.1918 - accuracy: 0.5339 - val_loss: 8.9575 - val_accuracy: 0.0337
Epoch 4/10
693/693 [==============================] - 5s 8ms/sample - loss: 1.3784 - accuracy: 0.7460 - val_loss: 10.5386 - val_accuracy: 0.0404
Epoch 5/10
693/693 [==============================] - 5s 8ms/sample - loss: 0.7475 - accuracy: 0.8903 - val_loss: 7.5199 - val_accuracy: 0.0572
Epoch 6/10
693/693 [==============================] - 6s 8ms/sample - loss: 0.4565 - accuracy: 0.9307 - val_loss: 5.1121 - val_accuracy: 0.1414
Epoch 7/10
693/693 [==============================] - 6s 8ms/sample - loss: 0.2489 - accu

{'loss': 2.7983851264221498, 'accuracy': 0.4107744}

In [126]:
test_preds = model.predict(X_train_ri).argmax(axis=1)
mlutils.plot_confusion_matrix(y_train_oh, test_preds, classes=np.r_([i for i in range(99)]), normalize=True)

AttributeError: module 'lib' has no attribute 'plot_confusion_matrix'

# Second model with Dropout

In [160]:
def get_conv_model(num_classes, img_size=32, compile=True):
    print("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    layers = tf.keras.layers.Conv2D(2,(3,3), activation="relu")(inputs) 
    layers = tf.keras.layers.BatchNormalization()(layers)
    layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.Dense(200, activation=tf.nn.relu)(layers)#change number of neurons of dense layer
    layers = tf.keras.layers.BatchNormalization()(layers)
    layers = tf.keras.layers.Dense(100, activation=tf.nn.relu)(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax, name="output_1")(layers)
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    if compile:
        model.compile(optimizer='RMSprop',loss='categorical_crossentropy' ,metrics=['accuracy'] )
    return model

In [161]:
#instantiate the model
num_classes = len( y_test_oh[1])
model1 = get_conv_model(num_classes,img_size=128)
model1.summary()

using 99 classes
Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 126, 126, 2)       20        
_________________________________________________________________
flatten_14 (Flatten)         (None, 31752)             0         
_________________________________________________________________
dense_28 (Dense)             (None, 200)               6350600   
_________________________________________________________________
dense_29 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
output_1 (Dense)             (None, 99)  

In [162]:
def train(model1, batch_size, epochs, model_name=""):
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/"+model_name+"_"+"{}".format(time()))
    model.reset_states()
    model.fit(X_train_ri.astype(np.float32), y_train_oh.astype(int), epochs=epochs, callbacks=[tensorboard], batch_size=batch_size, validation_data=( X_test_ri, y_test_oh))
    metrics = model.evaluate( X_test_ri.astype(np.float32), y_test_oh.astype(int))
    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [163]:
train(model1, batch_size=8, epochs=20, model_name="model_B")

Train on 693 samples, validate on 297 samples
Epoch 1/20
693/693 [==============================] - 5s 7ms/sample - loss: 0.0036 - accuracy: 0.9986 - val_loss: 4.9929 - val_accuracy: 0.4209
Epoch 2/20
693/693 [==============================] - 4s 6ms/sample - loss: 0.0066 - accuracy: 0.9986 - val_loss: 11.8920 - val_accuracy: 0.1448
Epoch 3/20
693/693 [==============================] - 4s 6ms/sample - loss: 0.0016 - accuracy: 1.0000 - val_loss: 4.4273 - val_accuracy: 0.4613
Epoch 4/20
693/693 [==============================] - 5s 8ms/sample - loss: 0.0020 - accuracy: 1.0000 - val_loss: 16.0824 - val_accuracy: 0.0842
Epoch 5/20
693/693 [==============================] - 6s 8ms/sample - loss: 0.0038 - accuracy: 0.9971 - val_loss: 6.0636 - val_accuracy: 0.3603
Epoch 6/20
693/693 [==============================] - 6s 8ms/sample - loss: 0.0022 - accuracy: 1.0000 - val_loss: 5.0114 - val_accuracy: 0.4512
Epoch 7/20
693/693 [==============================] - 6s 8ms/sample - loss: 0.0152 - acc

{'loss': 10.25324668306293, 'accuracy': 0.2053872}